# Tests

## Transcription

We are using OpenAI Whisper-1 model using OpenAI API.

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

# Ensure your OpenAI API key is available as an environment variable

if not os.getenv("OPENAI_API_KEY"):
    print("No OPENAI_API_KEY found")
    pass

AUDIO_PATH = Path("test_data/set2/200327_7822.mp3")
assert AUDIO_PATH.exists(), f"Audio file not found: {AUDIO_PATH}"


In [3]:
from openai import OpenAI

client = OpenAI()

# Transcribe with whisper-1
# Docs pattern: client.audio.transcriptions.create(model="whisper-1", file=...)
with open(AUDIO_PATH, "rb") as f:
    transcription = client.audio.transcriptions.create(
        model="whisper-1",
        file=f,
        language="fr",           
        temperature=0.0,
    )

print(type(transcription))
print(getattr(transcription, "text", transcription))


<class 'openai.types.audio.transcription.Transcription'>
Donc on va déjà commencer par l'allée, l'accès à la partie jardin située à l'arrière de l'habitation du requérant. Alors c'est quoi ? C'est des petites pierres, c'est quoi que vous avez mis ? C'est des petits graviers, donc une allée en gravier qui est en bon état, l'accès est assez, il n'y a pas d'ornières, il n'y a pas de trous, et puis tout est bien aplati. J'ai une démarcation, alors vous m'avez dit avec des petites bordures métalliques, un bordure métallique qui fait la séparation entre l'espace végétal, herbe, et puis l'accès en gravier. Donc là je vois, certaines sont couvertes par l'herbe, alors on les retrouve.


In [4]:
# Save transcript to a .txt next to the audio file
output_txt = AUDIO_PATH.with_suffix(".txt")
text = getattr(transcription, "text", None)

if text is None and isinstance(transcription, dict):
    text = transcription.get("text")

if not text:
    raise RuntimeError("No transcription text received.")

output_txt.write_text(text, encoding="utf-8")
print(f"Saved transcript to: {output_txt}")


Saved transcript to: test_data\set2\200327_7822.txt


## Formatting

This section formats a raw transcript with an OpenAI model using the instruction: "You are a professional notary making an observation."


In [5]:
from typing import Optional

# Reuse the existing OpenAI client if available; otherwise create one
try:
    client
except NameError:
    from openai import OpenAI
    client = OpenAI()

SYSTEM_INSTRUCTION = (
    "You are a professional notary making an observation. "
    "Rewrite and format the provided transcript into a clear, formal observation. "
    "- Keep factual, objective tone; avoid speculation.\n"
    "- Correct obvious transcription errors (names if confidently inferable; otherwise mark [inaudible] or [unintelligible]).\n"
    "- Use proper punctuation, paragraphs, and consistent tense.\n"
    "- Preserve dates, times, amounts, and legal-relevant details.\n"
    "- If content is in French, respond in French; otherwise respond in the source language."
)

def format_transcript(raw_text: str, *, model: str = "gpt-4o-mini") -> str:
    if not raw_text or not raw_text.strip():
        return ""

    # Prefer chat.completions for broad compatibility
    try:
        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": SYSTEM_INSTRUCTION},
                {"role": "user", "content": raw_text},
            ],
            temperature=0.2,
        )
        content: Optional[str] = None
        if resp and resp.choices and resp.choices[0].message:
            content = resp.choices[0].message.content
        return content or ""
    except Exception as e:
        raise RuntimeError(f"Formatting failed: {e}")



In [6]:
# Format the previously obtained transcript `text` and save next to the audio file
try:
    raw_text = text
except NameError:
    # If `text` isn't defined in this kernel, try to read from the .txt file next to AUDIO_PATH
    raw_text = ""
    try:
        raw_text = AUDIO_PATH.with_suffix(".txt").read_text(encoding="utf-8")
    except Exception:
        pass

if not raw_text:
    raise RuntimeError("No transcript text found. Run the transcription section first.")

formatted = format_transcript(raw_text)
print(formatted)

formatted_path = AUDIO_PATH.with_name(AUDIO_PATH.stem + "_formatted.txt")
formatted_path.write_text(formatted, encoding="utf-8")
print(f"Saved formatted text to: {formatted_path}")


Observation du  [date non spécifiée]

L'accès à la partie jardin située à l'arrière de l'habitation du requérant a été examiné. Il s'agit d'une allée en gravier, composée de petites pierres. L'état de cette allée est jugé satisfaisant, car il n'y a pas d'ornières ni de trous visibles. La surface est bien aplatie.

Une démarcation a été observée, constituée de bordures métalliques. Ces bordures servent de séparation entre l'espace végétal, recouvert d'herbe, et l'allée en gravier. Il a été noté que certaines bordures sont partiellement recouvertes par l'herbe, mais elles restent visibles.
Saved formatted text to: test_data\set2\200327_7822_formatted.txt


## Audio format conversion